In [1]:
from dotenv import load_dotenv
from google import genai
from google.genai.types import GenerateContentConfig, Tool, Part, Content
import os

_ = load_dotenv()

In [2]:
client = genai.Client(api_key=os.getenv("GOOGLE_GENAI_API_KEY"))

In [3]:
def get_weather(city: str) -> str:
    """
    Get the current weather for a given city.

    Args:
        city (str): The name of the city.
    Returns:
        str: A string describing the current weather.
    """
    
    # Complex logic to get real weather data would go here.
    # This includes:
    # - Constructing the API request
    # - Managing credentials and authentication
    # - Calling a weather API
    # - Handling errors and edge cases
    # - Parsing the response

    return f"The current weather in {city} is sunny with a temperature of 25°C."

In [4]:
# Define the function declaration for the model
get_weather_function = {
    "name": "get_weather",
    "description": "Gets the current weather for a specified city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to get the weather for.",
            },
        },
        "required": ["city"],
    },
}

In [5]:
tools = Tool(function_declarations=[get_weather_function])
config = GenerateContentConfig(tools=[tools])

In [ ]:
while True:
    user_message = input("You: ").strip()

    if user_message.lower() in {"exit", "quit"}:
        print("Agent: Goodbye! 👋")
        break

    if not user_message:
        continue

    # Define user prompt
    contents = [
        Content(
            role="user",
            parts=[Part(text=user_message)],
        )
    ]

    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=config,
            contents=contents,
        )
    except Exception as e:
        print(f"⚠️  Error talking to Gemini: {e}")
        continue

    # Check if the model wants to call a function
    if response.candidates[0].content.parts[0].function_call:
        tool_call = response.candidates[0].content.parts[0].function_call
        print(f"Function to call: {tool_call.name}")
        print(f"Arguments: {tool_call.args}")

        # Here we would parse the arguments and call the actual function
        if tool_call.name == "get_weather":
            weather_info = get_weather(**tool_call.args)

            function_response_part = Part.from_function_response(
                name=tool_call.name,
                response={"result": weather_info},
            )

            contents.append(response.candidates[0].content)
            contents.append(Content(role="user", parts=[function_response_part]))

            response = client.models.generate_content(
                model="gemini-2.0-flash",
                config=config,
                contents=contents,
            )

    agent_reply = response.candidates[0].content.parts[0].text

    print(f"User: {user_message}\n")
    print(f"Agent: {agent_reply}\n")

⚠️  Error talking to Gemini: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\nPlease retry in 30.697623311s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limi